In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# handle gpu leakage issue on DSI cluster
# ideally we shouldn't have to do this but leaving it for now in case the issue pops up again
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer

import scripts

In [7]:
# Confirm GPUs are working properly and set default device explicitly
device = "cuda:0"
torch.cuda.device_count()

1

In [4]:
MODEL_DIR = "/net/projects/veitch/LLMs/"
MODEL = "llama1-based-models/alpaca-7b" # or llama2-based-models/llama2-hf/Llama-2-7b-chat-hf
MODEL_PATH = MODEL_DIR + MODEL

In [5]:
# TODO: Clean this up

# This cell is for doing ROME edits
# ROME model and tokenizer
# MODEL_NAME = "gpt2-xl"
# model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
# tokenizer.pad_token = tokenizer.eos_token

In [8]:
tokenizer = transformers.LlamaTokenizer.from_pretrained(MODEL_PATH)
model = transformers.LlamaForCausalLM.from_pretrained(MODEL_PATH, low_cpu_mem_usage=True, torch_dtype=torch.float16, device_map="auto").to(device)

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [45]:
LABELS = "title" # choices are "title" and "categories"
DATA_FOLDER = "data/"
TRAIN_FILEPATH = DATA_FOLDER + "train-12-articles.csv"
VAL_FILEPATH = DATA_FOLDER + "val-12-articles.csv"
PROBE_TYPE = "linear" # choices are "linear" and "mlp"
LAYER = None
EPOCHS = 50
BATCH_SIZE = 4
AGGREGATION = "max" # choices are "max" or "mean"
PRINT_PROGRESS = True

In [46]:
train_data, label_encoder_title, label_encoder_cats = scripts.get_hidden_states(TRAIN_FILEPATH, model, tokenizer, device)
val_data, _, _ = scripts.get_hidden_states(VAL_FILEPATH, model, tokenizer, device)

In [63]:
train_dataset = scripts.create_dataset(*train_data, layer=None)

x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)
x.shape:  (135168,)


In [62]:
val_dataset = scripts.create_dataset(*val_data, layer=None)

hs.shape:  (20, 135168)
x.shape:  (135168,)
hs.shape:  (22, 135168)
x.shape:  (135168,)
hs.shape:  (14, 135168)
x.shape:  (135168,)
hs.shape:  (13, 135168)
x.shape:  (135168,)
hs.shape:  (15, 135168)
x.shape:  (135168,)
hs.shape:  (17, 135168)
x.shape:  (135168,)
hs.shape:  (66, 135168)
x.shape:  (135168,)
hs.shape:  (55, 135168)
x.shape:  (135168,)
hs.shape:  (37, 135168)
x.shape:  (135168,)
hs.shape:  (33, 135168)
x.shape:  (135168,)
hs.shape:  (19, 135168)
x.shape:  (135168,)
hs.shape:  (9, 135168)
x.shape:  (135168,)
hs.shape:  (36, 135168)
x.shape:  (135168,)
hs.shape:  (41, 135168)
x.shape:  (135168,)
hs.shape:  (29, 135168)
x.shape:  (135168,)
hs.shape:  (56, 135168)
x.shape:  (135168,)
hs.shape:  (23, 135168)
x.shape:  (135168,)
hs.shape:  (18, 135168)
x.shape:  (135168,)
hs.shape:  (44, 135168)
x.shape:  (135168,)
hs.shape:  (48, 135168)
x.shape:  (135168,)
hs.shape:  (41, 135168)
x.shape:  (135168,)
hs.shape:  (22, 135168)
x.shape:  (135168,)
hs.shape:  (19, 135168)
x.shape: 

In [67]:
# TODO: train_dataset isn't working here
scripts.train_handler(model, train_dataset, val_dataset, label_encoder_title, probe_type=PROBE_TYPE, labels=LABELS, batch_size=BATCH_SIZE, epochs=EPOCHS, print_progress=PRINT_PROGRESS)

ValueError: num_samples should be a positive integer value, but got num_samples=0